In [1]:
import pandas as pd
import pymongo
from bs4 import BeautifulSoup

In [3]:
artists = pd.read_csv('artists.csv')
display(artists.info())
artists.head()

C:\Users\Brynna\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466083 entries, 0 to 1466082
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   mbid              1466083 non-null  object 
 1   artist_mb         1466075 non-null  object 
 2   artist_lastfm     986756 non-null   object 
 3   country_mb        662368 non-null   object 
 4   country_lastfm    211498 non-null   object 
 5   tags_mb           119946 non-null   object 
 6   tags_lastfm       381075 non-null   object 
 7   listeners_lastfm  986760 non-null   float64
 8   scrobbles_lastfm  986760 non-null   float64
 9   ambiguous_artist  1466083 non-null  bool   
dtypes: bool(1), float64(2), object(7)
memory usage: 102.1+ MB


None

mbid              artist_mb  \
0  cc197bad-dc9c-440d-a5b5-d52ba2e14234               Coldplay   
1  a74b1b7f-71a5-4011-9441-d0b5e4122711              Radiohead   
2  8bfac288-ccc5-448d-9573-c33ea2aa5c30  Red Hot Chili Peppers   
3  73e5e69d-3554-40d8-8516-00cb38737a1c                Rihanna   
4  b95ce3ff-3d05-4e87-9e01-c97b66af13d4                 Eminem   

           artist_lastfm      country_mb           country_lastfm  \
0               Coldplay  United Kingdom           United Kingdom   
1              Radiohead  United Kingdom           United Kingdom   
2  Red Hot Chili Peppers   United States            United States   
3                Rihanna   United States  Barbados; United States   
4                 Eminem   United States            United States   

                                             tags_mb  \
0  rock; pop; alternative rock; british; uk; brit...   
1  rock; electronic; alternative rock; british; g...   
2  rock; alternative rock; 80s; 90s; rap; metal; ...   
3  pop; dance; hip hop; reggae; contemporary r b;...   
4  turkish; rap; american; hip-hop; hip hop; hiph...   

                                         tags_lastfm  listeners_lastfm  \
0  rock; alternative; britpop; alternative rock; ...         5381567.0   
1  alternative; alternative rock; rock; indie; el...         4732528.0   
2  rock; alternative rock; alternative; Funk Rock...         4620835.0   
3  pop; rnb; female vocalists; dance; Hip-Hop; Ri...         4558193.0   
4  rap; Hip-Hop; Eminem; hip hop; pop; american; ...         4517997.0   

   scrobbles_lastfm  ambiguous_artist  
0       360111850.0             False  
1       499548797.0             False  
2       293784041.0             False  
3       199248986.0             False  
4       199507511.0             False

In [4]:
new_artists = artists[['artist_lastfm', 'country_lastfm', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm']]
new_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466083 entries, 0 to 1466082
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist_lastfm     986756 non-null  object 
 1   country_lastfm    211498 non-null  object 
 2   tags_lastfm       381075 non-null  object 
 3   listeners_lastfm  986760 non-null  float64
 4   scrobbles_lastfm  986760 non-null  float64
dtypes: float64(2), object(3)
memory usage: 55.9+ MB


In [5]:
clean_artists = new_artists.dropna(how= 'any')
clean_artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204418 entries, 0 to 986728
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist_lastfm     204418 non-null  object 
 1   country_lastfm    204418 non-null  object 
 2   tags_lastfm       204418 non-null  object 
 3   listeners_lastfm  204418 non-null  float64
 4   scrobbles_lastfm  204418 non-null  float64
dtypes: float64(2), object(3)
memory usage: 9.4+ MB


In [6]:
clean_artists = clean_artists.rename(columns= {'artist_lastfm': 'Artist', 'country_mb': 'Country', 'tags_lastfm': 'Genre', 'listeners_lastfm': 'Listener Count', 'scrobbles_lastfm': 'Play Count'})

In [6]:
clean_artists['Genre'] = clean_artists['Genre'].str.split('seen live; ', expand= True)[0].str.split(';', expand= True)[0]
clean_artists.head()

Artist           country_lastfm        Genre  \
0               Coldplay           United Kingdom         rock   
1              Radiohead           United Kingdom  alternative   
2  Red Hot Chili Peppers            United States         rock   
3                Rihanna  Barbados; United States          pop   
4                 Eminem            United States          rap   

   Listener Count   Play Count  
0       5381567.0  360111850.0  
1       4732528.0  499548797.0  
2       4620835.0  293784041.0  
3       4558193.0  199248986.0  
4       4517997.0  199507511.0

In [7]:
clean_artists['Genre'] = clean_artists['Genre'].str.replace('seen live;', '')

In [8]:
clean_artists['Genre'] = clean_artists['Genre'].str.split(';', expand= True)[0]
clean_artists.head()

Artist           country_lastfm        Genre  \
0               Coldplay           United Kingdom         rock   
1              Radiohead           United Kingdom  alternative   
2  Red Hot Chili Peppers            United States         rock   
3                Rihanna  Barbados; United States          pop   
4                 Eminem            United States          rap   

   Listener Count   Play Count  
0       5381567.0  360111850.0  
1       4732528.0  499548797.0  
2       4620835.0  293784041.0  
3       4558193.0  199248986.0  
4       4517997.0  199507511.0

In [7]:
clean_artists.drop_duplicates(subset= 'Artist', keep = 'first', inplace = True)


In [9]:
clean_artists['Genre'].value_counts()

black metal              5985
electronic               5320
pop                      4600
rock                     4600
folk                     4327
                         ... 
rock chile                  1
 brighton                   1
spanish impressionism       1
comicfr                     1
nova trova                  1
Name: Genre, Length: 6002, dtype: int64

In [10]:
clean_artists.to_csv(r'clean_artists2.csv', index=False, header= True)

In [7]:
billboard = pd.read_csv('Billboard.csv')
display(billboard.info())
billboard.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951 entries, 0 to 1950
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      1951 non-null   int64 
 1   Song            1951 non-null   object
 2   Artist          1951 non-null   object
 3   Weeks On #1     1951 non-null   int64 
 4   Weeks On Chart  1951 non-null   int64 
 5   Peak Rank       1951 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 91.6+ KB


None

Unnamed: 0               Song                            Artist  \
0           0        Blank Space                      Taylor Swift   
1           1  Take Me To Church                            Hozier   
2           2       Uptown Funk!  Mark Ronson Featuring Bruno Mars   
3           3  Thinking Out Loud                        Ed Sheeran   
4           4     Lips Are Movin                    Meghan Trainor   

   Weeks On #1  Weeks On Chart  Peak Rank  
0            7              36          1  
1            0              41          2  
2           14              56          1  
3            0              58          2  
4            0              29          4

In [8]:
new_billboard = billboard[['Artist', 'Weeks On #1', 'Weeks On Chart', 'Peak Rank']]
new_billboard.head()

Artist  Weeks On #1  Weeks On Chart  Peak Rank
0                      Taylor Swift            7              36          1
1                            Hozier            0              41          2
2  Mark Ronson Featuring Bruno Mars           14              56          1
3                        Ed Sheeran            0              58          2
4                    Meghan Trainor            0              29          4

In [9]:
new_billboard['Artist'].value_counts()

Drake                                      64
The Weeknd                                 26
Future                                     25
J. Cole                                    25
XXXTENTACION                               22
                                           ..
Calvin Harris Featuring Future & Khalid     1
Paul McCartney                              1
Ty Dolla $ign Featuring E-40                1
Lil Wayne Featuring XXXTENTACION            1
The Chainsmokers Featuring Emily Warren     1
Name: Artist, Length: 1072, dtype: int64

In [10]:
new_billboard[['Artist', 'Featuring']] = new_billboard.Artist.str.split("Featuring", expand= True)
new_billboard.head()

C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Artist  Weeks On #1  Weeks On Chart  Peak Rank    Featuring
0    Taylor Swift            7              36          1         None
1          Hozier            0              41          2         None
2    Mark Ronson            14              56          1   Bruno Mars
3      Ed Sheeran            0              58          2         None
4  Meghan Trainor            0              29          4         None

In [11]:
billboard_group = new_billboard.groupby(['Artist', 'Featuring']).sum()
billboard_group

Weeks On #1  \
Artist                            Featuring                                              
2 Chainz                           Lil Wayne                                         0   
                                   Travis Scott                                      0   
                                   Ty Dolla $ign, Trey Songz & Jhene Aiko            0   
                                   YG & Offset                                       0   
21 Savage, Offset & Metro Boomin   Quavo                                             0   
...                                                                                ...   
Zedd                               Selena Gomez                                      0   
Zendaya                            Chris Brown                                       0   
benny blanco & Juice WRLD          Brendon Urie                                      0   
gnash                              Johnny Yukon                                      0   
                                   Olivia O'Brien                                    0   

                                                                           Weeks On Chart  \
Artist                            Featuring                                                 
2 Chainz                           Lil Wayne                                            0   
                                   Travis Scott                                        12   
                                   Ty Dolla $ign, Trey Songz & Jhene Aiko              20   
                                   YG & Offset                                          2   
21 Savage, Offset & Metro Boomin   Quavo                                                3   
...                                                                                   ...   
Zedd                               Selena Gomez                                        16   
Zendaya                            Chris Brown                                          3   
benny blanco & Juice WRLD          Brendon Urie                                        31   
gnash                              Johnny Yukon                                         5   
                                   Olivia O'Brien                                      39   

                                                                           Peak Rank  
Artist                            Featuring                                           
2 Chainz                           Lil Wayne                                     100  
                                   Travis Scott                                   55  
                                   Ty Dolla $ign, Trey Songz & Jhene Aiko         44  
                                   YG & Offset                                    96  
21 Savage, Offset & Metro Boomin   Quavo                                          64  
...                                                                              ...  
Zedd                               Selena Gomez                                   17  
Zendaya                            Chris Brown                                    92  
benny blanco & Juice WRLD          Brendon Urie                                    6  
gnash                              Johnny Yukon                                   74  
                                   Olivia O'Brien                                 10  

[512 rows x 3 columns]

In [12]:
spotify = pd.read_csv('spotify_top_tens.csv', encoding='ISO-8859-1')
spotify.head()

Unnamed: 0                 title      artist        top genre  year  bpm  \
0           1      Hey, Soul Sister       Train       neo mellow  2010   97   
1           2  Love The Way You Lie      Eminem  detroit hip hop  2010   87   
2           3               TiK ToK       Kesha        dance pop  2010  120   
3           4           Bad Romance   Lady Gaga        dance pop  2010  119   
4           5  Just the Way You Are  Bruno Mars              pop  2010  109   

   nrgy  dnce  dB  live  val  dur  acous  spch  pop  
0    89    67  -4     8   80  217     19     4   83  
1    93    75  -5    52   64  263     24    23   82  
2    84    76  -3    29   71  200     10    14   80  
3    92    70  -4     8   71  295      0     4   79  
4    84    64  -5     9   43  221      2     4   78

In [13]:
clean_spotify = spotify.rename(columns = {'artist': 'Artist', 'top genre': 'Genre'})
clean_spotify = clean_spotify[['Artist', 'Genre']]
clean_spotify.head()

Artist            Genre
0       Train       neo mellow
1      Eminem  detroit hip hop
2       Kesha        dance pop
3   Lady Gaga        dance pop
4  Bruno Mars              pop

Katy Perry         17
Justin Bieber      16
Maroon 5           15
Rihanna            15
Lady Gaga          14
                   ..
Icona Pop           1
Jonas Blue          1
Joey Montana        1
Jennifer Hudson     1
Jewel               1
Name: Artist, Length: 184, dtype: int64